# BMO Vancouver Marathon
# Year over year comparison and analysis

Special Olympics BC has been a charity partner for the BMO Vancouver Marathon since 2022. Over the years we have increased our fundraising efforts to grow this event which has yielded good results. The following are data on our past three events to support planning and decision making for the 2026 event.

In [1]:
import numpy as np
import altair as alt
import pandas as pd
import seaborn as sns

Import the data from excel

In [2]:
donations = pd.read_excel('data/bmo 2023-2025.xlsx', sheet_name= 'Donations - All time')
runners = pd.read_excel('data/bmo 2023-2025.xlsx', sheet_name= 'Runners - All time')
#donations
#runners

## Cumulative donation year over year

The graph below show the cumulative donation from the past three event whether its a general donation or donation to a runner. The 2024 and 2025 year show a jump closer to the event date.

In [3]:
line_chart = alt.Chart(donations).transform_calculate(
    shifted_date=alt.expr.toDate(
        alt.expr.datetime(
            alt.expr.year(alt.datum.Date),
            alt.expr.month(alt.datum.Date) + 7,
            alt.expr.date(alt.datum.Date) +20
        )
    )
).transform_window(
    cumulative_donation='sum(Donated)',
    sort=[{'field': 'shifted_date'}], # Sort by the shifted date
    groupby=['Event Year']
).mark_line(strokeWidth=2).encode(
    alt.X('monthdate(shifted_date):T', title='Month-Day (Shifted +6 Months)', axis=None),
    alt.Y('cumulative_donation:Q', title='Cumulative Donation'), color='Event Year:N',
    tooltip='Event Year:N'
).properties(width=500)

line_chart 

alt.Chart(...)

## Donor behaviour, donation to a runner vs general donation
Lets see the donation activity between donors that donated in general and donors that donated to a fundraiser. 
Interestingly enough, in 2023 almost all donations were general donations. But, in 2025 our runners seem to be the main driver for revenue. With donation triple of general donation. Whereas in 2024 general donation and donation to a fundraiser does not show too much gap.

General donation shows a steady growth leading up to race day.

In [4]:
# Chart for 'Fundraiser first name' is null
null_chart = alt.Chart(donations).transform_filter(
    alt.datum['Fundraiser first name'] == None  # Filter for null
).transform_calculate(
    shifted_date=alt.expr.toDate(
        alt.expr.datetime(
            alt.expr.year(alt.datum.Date),
            alt.expr.month(alt.datum.Date) + 7,
            alt.expr.date(alt.datum.Date) + 20
        )
    )
).transform_window(
    cumulative_donation='sum(Donated)',
    sort=[{'field': 'shifted_date'}],  # Sort by the shifted date
    groupby=['Event Year']
).mark_line(strokeWidth=2).encode(
    alt.X('monthdate(shifted_date):T', title='Month-Day (Shifted +6 Months)', axis=None),
    alt.Y('cumulative_donation:Q', title='Cumulative Donation',
          scale=alt.Scale(domain=[0, 3500])),  # Set fixed Y-axis range
    color='Event Year:N',
    tooltip='Event Year:N'
).properties(
    title="Cumulative Donations (Fundraiser First Name: Null)",
    width=500
)

# Chart for 'Fundraiser first name' is not null
not_null_chart = alt.Chart(donations).transform_filter(
    alt.datum['Fundraiser first name'] != None  # Filter for not null
).transform_calculate(
    shifted_date=alt.expr.toDate(
        alt.expr.datetime(
            alt.expr.year(alt.datum.Date),
            alt.expr.month(alt.datum.Date) + 7,
            alt.expr.date(alt.datum.Date) + 20
        )
    )
).transform_window(
    cumulative_donation='sum(Donated)',
    sort=[{'field': 'shifted_date'}],  # Sort by the shifted date
    groupby=['Event Year']
).mark_line(strokeWidth=2).encode(
    alt.X('monthdate(shifted_date):T', title='Month-Day (Shifted +6 Months)', axis=None),
    alt.Y('cumulative_donation:Q', title='Cumulative Donation',
          scale=alt.Scale(domain=[0, 3500])),  # Set fixed Y-axis range
    color='Event Year:N',
    tooltip='Event Year:N'
).properties(
    title="Cumulative Donations (Fundraiser First Name: Not Null)",
    width=500
)

# Combine the two charts horizontally
two_charts = alt.hconcat(null_chart, not_null_chart).resolve_scale(
    y='shared'  # Share the Y-axis scale between the charts
)

two_charts

alt.HConcatChart(...)

Let's investigate if knowing someone in the race affects donor behaviour.

Below is the comparison between average donation to a runner and average general donation. Donation to a runner is generally triple than general donation.

In [5]:
# Step 1: Add a helper column indicating whether 'Field' is null or not
donations['Field Status'] = np.where(donations['Fundraiser first name'].isnull(), 'Not to a Runner', 'To a Runner')

# Step 2: Create the Altair chart
donor_behaviour_chart = alt.Chart(donations).mark_bar().encode(
    x=alt.X('Field Status:N', title='Field Status',axis=alt.Axis(labelAngle=0)),  # 'N' indicates nominal (categorical) data
    y=alt.Y('average(Donated):Q', 
            title='Average Donation',
            axis=alt.Axis(format='$,.0f')  # Format as dollar values
    ),
    color=alt.Color('Field Status:N', legend=None)  # Optional: Color bars by status
).properties(
    title='Comparison of Average Donation by Field Status',
    width=400
)

# Display the chart
donor_behaviour_chart

alt.Chart(...)

Although there aren't clear indication on which area drives revenue, we should continue to invite people to donate while also supporting our runners in their fundraising journey.

## Other year over year comparison
### Number of runners year over year

In [6]:
reg_chart = alt.Chart(runners).mark_bar().encode(
    x=alt.X('Event Year:O', title='Event',axis=alt.Axis(labelAngle=0)),  # 'O' indicates ordinal data
    y=alt.Y('count()', title='Count')            # Use count aggregation for Y-axis
).properties(
    title='Runners by Event Year', width=500
)

num_of_runners_labels = reg_chart.mark_text(
    align='center',
    baseline='middle',
    dy=+15,  # Adjust text position above the bar
    color='white'
).encode(
    text=alt.Text('count()')  # Format labels as dollar values
)

# Display the charts
reg_chart + num_of_runners_labels

alt.LayerChart(...)

### Average raised year over year

In [7]:
avg_raised = alt.Chart(runners).mark_bar().encode(
    x=alt.X('Event Year:O', title='Event',axis=alt.Axis(labelAngle=0)),  # 'O' indicates ordinal data
    y=alt.Y('average(Total amount raised)', 
            title='Average Amount Raised',
            axis=alt.Axis(format='$,.0f')    # Format as dollar values, rounded to nearest dollar
    )
).properties(
    title='Avg Raised by Event Year', 
    width=500
)

labels = avg_raised.mark_text(
    align='center',
    baseline='middle',
    dy=+15,  # Adjust text position above the bar
    color='white'
).encode(
    text=alt.Text('average(Total amount raised):Q', format='$,.0f')  # Format labels as dollar values
)

# Combine the bar chart and labels
avg_raised + labels


alt.LayerChart(...)

### Average donation year over year

In [8]:
avg_donated = alt.Chart(donations).mark_bar().encode(
    x=alt.X('Event Year:O', title='Event',axis=alt.Axis(labelAngle=0)),  # 'O' indicates ordinal data
    y=alt.Y('average(Donated)', 
        title='Average Donation',
        axis=alt.Axis(format='$,.0f')    # Remove unnecessary padding
)
).properties(
    title='Avg Donation by Event Year', 
    width=500
)

avgdonatedlabels = avg_donated.mark_text(
    align='center',
    baseline='middle',
    dy=+15,  # Adjust text position above the bar
    color='white'
).encode(
    text=alt.Text('average(Donated):Q', format='$,.0f')  # Format labels as dollar values
)

# Combine the bar chart and labels
avg_donated + avgdonatedlabels


alt.LayerChart(...)